## Working with LangGraph

In [1]:
# loading neccessary libraries
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image,display
import os 

print("✅ All imports successful")


✅ All imports successful


In [2]:
# initialize the LLM
llm= ChatOpenAI(model="gpt-4o-mini",temperature=0.7)
print(f"✅ LLM initialized: {llm.model_name}")


✅ LLM initialized: gpt-4o-mini


In [3]:
# System prompt that defines assistance behavior
sys_msg=SystemMessage(
    content="You are a friendly assistant that answers user questions. Be helpful and concise.")
def assistance(state: MessagesState) -> dict:
    """
    The assistance node- process message and generate response
    """
    # Combine the system prompt with conversation history
    messages= [sys_msg]+ state["messages"]

    # Get response from llm
    response= llm.invoke(messages)

    # return as state update
    return {"messages":[AIMessage(content=response.content)]}
print("✅ Assistant node defined")

✅ Assistant node defined


### Building the Graph

In [4]:
# Create a StateGraph with MessageState
builder = StateGraph(MessagesState)

# Add the assistance node
builder.add_node("assistance", assistance)

# Now let define the flow:
# START → assistance → END
builder.add_edge(START, "assistance")
builder.add_edge("assistance", END)

print("✅ Graph structure defined")

✅ Graph structure defined


In [5]:
# Adding memory
memory=MemorySaver()

# Compile the graph with memeory
agent= builder.compile(checkpointer=memory)
print("✅ Agent compiled with memory")

✅ Agent compiled with memory


In [6]:
# Let visualize our graph structure
try:
    display(Image(agent.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"Could not display graph: {e}")
    print("Graph structure: START → assistant → END")

Could not display graph: Failed to reach https://mermaid.ink API while trying to render your graph after 1 retries. To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`
Graph structure: START → assistant → END


In [7]:
# let create a session id and test run our agent
session_id= "user001"
print(f"Starting conversation with session ID: {session_id}")

Starting conversation with session ID: user001


In [8]:
# Helper function for conversatios
def run_chat(user_input:str, thread_id:str = session_id):
    """Send a message to the agent and get response."""
    # Invoke the agent
    result= agent.invoke({"messages":[HumanMessage(content=user_input)]},
    config={"configurable":{"thread_id":thread_id}})

    # print the conversation 
    for message in result["messages"]:
        if isinstance(message, HumanMessage):
             print(f"\n👤 User: {message.content}")
        elif isinstance(message, AIMessage):
            print(f"🤖 Agent: {message.content}")

    print("\n" + "="*70)
print("✅ Conversation function ready")


✅ Conversation function ready


In [ ]:
# Test the agent
run_chat("Hello, who are you?")

#### Testing if it remember conversation

In [ ]:
# First message
run_chat("My name is Saheed Olayinka Olayemi, popularly know as Olasquare")

In [ ]:
# Second message
run_chat("My name is ?")


👤 User: Hello, who are you?
🤖 Agent: Hello! I'm your friendly assistant, here to help you with any questions or information you need. How can I assist you today?

👤 User: My name is Saheed Olayinka Olayemi, popularly know as Olasquare
🤖 Agent: Nice to meet you, Olasquare! How can I assist you today?

👤 User: My name is ?
🤖 Agent: Your name is Saheed Olayinka Olayemi, but you are popularly known as Olasquare. How can I help you further?



In [ ]:
# Start a new topic
run_chat("I'm learning about RAG systems")


👤 User: Hello, who are you?
🤖 Agent: Hello! I'm your friendly assistant, here to help you with any questions or information you need. How can I assist you today?

👤 User: My name is Saheed Olayinka Olayemi, popularly know as Olasquare
🤖 Agent: Nice to meet you, Olasquare! How can I assist you today?

👤 User: My name is ?
🤖 Agent: Your name is Saheed Olayinka Olayemi, but you are popularly known as Olasquare. How can I help you further?

👤 User: I'm learning about RAG systems
🤖 Agent: That's great! RAG systems, or Retrieval-Augmented Generation systems, combine retrieval techniques with generative models to enhance the quality of responses. They retrieve relevant information from a database or knowledge base and then use that information to generate more accurate and contextually relevant responses. 

If you have specific questions about RAG systems or if there's a particular aspect you'd like to learn more about, feel free to ask!



In [ ]:
# Reference it
run_chat("Can you explain the main components?")


👤 User: Hello, who are you?
🤖 Agent: Hello! I'm your friendly assistant, here to help you with any questions or information you need. How can I assist you today?

👤 User: My name is Saheed Olayinka Olayemi, popularly know as Olasquare
🤖 Agent: Nice to meet you, Olasquare! How can I assist you today?

👤 User: My name is ?
🤖 Agent: Your name is Saheed Olayinka Olayemi, but you are popularly known as Olasquare. How can I help you further?

👤 User: I'm learning about RAG systems
🤖 Agent: That's great! RAG systems, or Retrieval-Augmented Generation systems, combine retrieval techniques with generative models to enhance the quality of responses. They retrieve relevant information from a database or knowledge base and then use that information to generate more accurate and contextually relevant responses. 

If you have specific questions about RAG systems or if there's a particular aspect you'd like to learn more about, feel free to ask!

👤 User: Can you explain the main components?
🤖 Agent:

In [ ]:
# Follow-up question
run_chat("Which component is most important?")


👤 User: Hello, who are you?
🤖 Agent: Hello! I'm your friendly assistant, here to help you with any questions or information you need. How can I assist you today?

👤 User: My name is Saheed Olayinka Olayemi, popularly know as Olasquare
🤖 Agent: Nice to meet you, Olasquare! How can I assist you today?

👤 User: My name is ?
🤖 Agent: Your name is Saheed Olayinka Olayemi, but you are popularly known as Olasquare. How can I help you further?

👤 User: I'm learning about RAG systems
🤖 Agent: That's great! RAG systems, or Retrieval-Augmented Generation systems, combine retrieval techniques with generative models to enhance the quality of responses. They retrieve relevant information from a database or knowledge base and then use that information to generate more accurate and contextually relevant responses. 

If you have specific questions about RAG systems or if there's a particular aspect you'd like to learn more about, feel free to ask!

👤 User: Can you explain the main components?
🤖 Agent:

### Let's test that different thread IDs have separate memories:

In [ ]:
# Conversation 1
print("\n🔵 CONVERSATION 1")
run_chat("My name is Saheed", thread_id="user_saheed")

# Conversation 2 (different user)
print("\n🟢 CONVERSATION 2")
run_chat("My name is Olayemi", thread_id="user_ola")


🔵 CONVERSATION 1

👤 User: My name is Saheed
🤖 Agent: Nice to meet you, Saheed! How can I assist you today?


🟢 CONVERSATION 2

👤 User: My name is Olayemi
🤖 Agent: Nice to meet you, Olayemi! How can I assist you today?



In [ ]:
# Back to Saheed - does it remember his name?
print("\n🔵 BACK TO CONVERSATION 1")
run_chat("What's my name?", thread_id="user_saheed")


🔵 BACK TO CONVERSATION 1

👤 User: My name is Saheed
🤖 Agent: Nice to meet you, Saheed! How can I assist you today?

👤 User: What's my name?
🤖 Agent: Your name is Saheed. How can I help you today?



In [ ]:
# Back to Olayemi - does it remember his name?
print("\n🔵 BACK TO CONVERSATION 2")
run_chat("What's my name?", thread_id="user_ola")


🔵 BACK TO CONVERSATION 2

👤 User: My name is Olayemi
🤖 Agent: Nice to meet you, Olayemi! How can I assist you today?

👤 User: What's my name?
🤖 Agent: Your name is Olayemi. How can I help you today?

